Source:
Github of Isabel Segura Bedmar 
https://github.com/isegura/BasicNLP

- SpaCy_NLP.ipynb (Clase 3)
- IntroNER_spacy.ipynb (Clase 6)
- Dictionary_based_NER_(spacy).ipynb (Clase 6)

## Connect Drive


In [0]:
from google.colab import drive
drive.mount("/content/drive/")
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
drive  sample_data


# Import libraries

In [0]:
#dataframes
import pandas as pd
from tabulate import tabulate

#regex
import re

#models
import spacy

#Input

###Input from source

https://zenodo.org/record/3693570#.Xo-JAfFS8_-   

version 1.0 10.5281/zenodo.3625747

3. Codes, Diagnostics, train/dev -- codiesp_codes/codiesp-D_codes.tsv
4. Annotations, Diagnostics, train -- train_annotations_codiespD_processed.tsv
5. Annotations, Diagnostics, dev -- dev_annotations_codiespD_processed.tsv
6. Codes, Procedures, train/dev -- codiesp_codes/codiesp-P_codes.tsv
7. Annotations, Procedures, train -- train_annotations_codiespP_processed.tsv
8. Annotations, Procedures, dev -- dev_annotations_codiespP_processed.tsv

###Input from other Notebook

In the notebook **"0904_terminado_Crear tsv a partir de text files.ipynb"**, there is a section called **"Create a tsv from all text files (train and dev)"**. It is necessary to execute it before load the independent text files.

https://colab.research.google.com/drive/1JhAqsmZBLJzpb0lA8BlU2Kzldv6WQ0b5#scrollTo=DLKJlTsxQ4YB

1. Articles, Diagnostics/Procedures, train -- train_dev/train/text_files/... --> train_dev/train/train_articles_codiesp.csv
2. Articles, Diagnostics/Procedures, dev -- train_dev/dev/text_files/... --> train_dev/dev/dev_articles_codiesp.csv


###Total input




1. Articles, Diagnostics/Procedures, train -- train_dev/train/train_articles_codiesp.csv
2. Articles, Diagnostics/Procedures, dev -- train_dev/dev/dev_articles_codiesp.csv
3. Codes, Diagnostics, train/dev -- codiesp_codes/codiesp-D_codes.tsv
4. Annotations, Diagnostics, train -- train_annotations_codiespD_processed.tsv
5. Annotations, Diagnostics, dev -- dev_annotations_codiespD_processed.tsv
6. Codes, Procedures, train/dev -- codiesp_codes/codiesp-P_codes.tsv
7. Annotations, Procedures, train -- train_annotations_codiespP_processed.tsv
8. Annotations, Procedures, dev -- dev_annotations_codiespP_processed.tsv

#Load data

1. Articles, Diagnostics/Procedures, train
2. Articles, Diagnostics/Procedures, dev
3. Codes, Diagnostics, train/dev
4. Annotations, Diagnostics, train
5. Annotations, Diagnostics, dev
6. Codes, Procedures, train/dev
7. Annotations, Procedures, train
8. Annotations, Procedures, dev

##Load articles from tsv (train and test)

In [0]:
def load_articles(path):
  df = pd.read_csv(path, sep=",", engine="python", encoding='utf-8')
  print("--- Loaded dataset:", path)

  print("--- Number of rows is " + str(df.shape[0]) + " x number of columns " + str(df.shape[1]))
  #print(df.head())
  #print(tabulate(df.head(), headers='keys', tablefmt='psql'))

  return df

path_codiesp='/content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/'

print("- Load 1. Articles, Diagnostics/Procedures, train")
train_file_articles = path_codiesp + 'train_dev/train/train_articles_codiesp.csv'
df_train_articles = load_articles(train_file_articles)

print("- Load 2. Articles, Diagnostics/Procedures, dev")
dev_file_articles = path_codiesp + 'train_dev/dev/dev_articles_codiesp.csv'
df_dev_articles = load_articles(dev_file_articles)

- Load 1. Articles, Diagnostics/Procedures, train
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/train/train_articles_codiesp.csv
--- Number of rows is 500 x number of columns 2
- Load 2. Articles, Diagnostics/Procedures, dev
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/dev/dev_articles_codiesp.csv
--- Number of rows is 250 x number of columns 2


##Loading annotations and codes from tsv (train and dev, D and P)

In [0]:
def load_data(path, names):
  df = pd.read_csv(path, sep="\t", names=names)
  print("--- Loaded dataset:", path)

  print("--- Number of rows is " + str(df.shape[0]) + " x number of columns " + str(df.shape[1]))
  #print(df.head())
  #print(tabulate(df.head(), headers='keys', tablefmt='psql'))

  return df

path_codiesp='/content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/'
names_annotations=["ArticleID", "Label", "ICD10-code", "Text-reference"]  #Columns: articleID \t label \t ICD10-code \t text-reference
names_codeD=["Code", "Es-description", "En-description"]   # Columns: code \t es-description \t en-description
names_codeP=["Code", "Es-description", "En-description", "Approach?"]   # Columns: code \t es-description \t en-description \t approach?

print("- Load 3. Codes, Diagnostics, train/dev")
file_codeD = path_codiesp + 'codiesp_codes/codiesp-D_codes.tsv'
df_codeD = load_data(file_codeD, names_codeD)

print("- Load 4. Annotations, Diagnostics, train")
file_train_annotationsD = path_codiesp + 'train_dev/train/train_annotations_codiespD_processed.tsv'
df_train_annotationsD = load_data(file_train_annotationsD, names_annotations)

print("- Load 5. Annotations, Diagnostics, dev")
file_dev_annotationsD = path_codiesp + 'train_dev/dev/dev_annotations_codiespD_processed.tsv'
df_dev_annotationsD = load_data(file_dev_annotationsD, names_annotations)

print("- Load 6. Codes, Procedures, train/dev")
file_codeP = path_codiesp + 'codiesp_codes/codiesp-P_codes.tsv'
df_codeP = load_data(file_codeP, names_codeP)

print("- Load 7. Annotations, Procedures, train")
file_train_annotationsP = path_codiesp + 'train_dev/train/train_annotations_codiespP_processed.tsv'
df_train_annotationsP = load_data(file_train_annotationsP, names_annotations)

print("- Load 8. Annotations, Procedures, dev")
file_dev_annotationsP = path_codiesp + 'train_dev/dev/dev_annotations_codiespP_processed.tsv'
df_dev_annotationsP = load_data(file_dev_annotationsP, names_annotations)

- Load 3. Codes, Diagnostics, train/dev
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/codiesp_codes/codiesp-D_codes.tsv
--- Number of rows is 71486 x number of columns 3
- Load 4. Annotations, Diagnostics, train
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/train/train_annotations_codiespD_processed.tsv
--- Number of rows is 7524 x number of columns 4
- Load 5. Annotations, Diagnostics, dev
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/dev/dev_annotations_codiespD_processed.tsv
--- Number of rows is 3589 x number of columns 4
- Load 6. Codes, Procedures, train/dev
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/codiesp_codes/codiesp-P_codes.tsv
--- Number of rows is 87170 x number of columns 4
- Load 7. Annotations, Procedures, train
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/train/train_annot

##Print dataframes for all the data

1. Articles, Diagnostics/Procedures, train
2. Articles, Diagnostics/Procedures, dev
3. Codes, Diagnostics, train/dev
4. Annotations, Diagnostics, train
5. Annotations, Diagnostics, dev
6. Codes, Procedures, train/dev
7. Annotations, Procedures, train
8. Annotations, Procedures, dev



In [0]:
print("- Print 1. Articles, Diagnostics/Procedures, train")
print(tabulate(df_train_articles.head(), headers='keys', tablefmt='psql'))

print("- Print 2. Articles, Diagnostics/Procedures, dev")
print(tabulate(df_dev_articles.head(), headers='keys', tablefmt='psql'))

print("- Print 3. Codes, Diagnostics, train/dev")
print(tabulate(df_codeD.head(), headers='keys', tablefmt='psql'))

print("- Print 4. Annotations, Diagnostics, train")
print(tabulate(df_train_annotationsD.head(), headers='keys', tablefmt='psql'))

print("- Print 5. Annotations, Diagnostics, dev")
print(tabulate(df_dev_annotationsD.head(), headers='keys', tablefmt='psql'))

print("- Print 6. Codes, Procedures, train/dev")
print(tabulate(df_codeP.head(), headers='keys', tablefmt='psql')) 

print("- Print 7. Annotations, Procedures, train")
print(tabulate(df_train_annotationsP.head(), headers='keys', tablefmt='psql'))   

print("- Print 8. Annotations, Procedures, dev")
print(tabulate(df_dev_annotationsP.head(), headers='keys', tablefmt='psql'))

- Print 1. Articles, Diagnostics/Procedures, train
+----+---------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Spacy functions

Spacy is a Python library that allows us to perform many NLP tasks such as
 
 
    - Tokenization
    - Pos Tagging
    - Entity Detection
    - Dependency Parsing
    - Semantic Similarity

 
 Another important advantages of  Spacy is that it can be used to generated word vectors (word embeddings) for every word in a sentence. Spacy already includes some of these pre-trained models, which you can use right now!!.
 

 https://nlpforhackers.io/complete-guide-to-spacy/
 
  

Spacy token attributes https://spacy.io/api/token

Spacy NER https://spacy.io/usage/linguistic-features

###Install Spacy and the model in spanish.

Using displaCy in Jupyter notebooks. Reference in : https://spacy.io/usage/visualizers

For dictionary based NER, it is necessary spacy-lookup library.

spacy-lookup library:   Github.

https://github.com/mpuig/spacy-lookup

Notebook.

https://colab.research.google.com/drive/1Xsne2tMz2IDk0X2Gn-9GIUcVDpAfS3-W?authuser=1#scrollTo=MR_FqfiQ5cLx


##Preprocessing

In [0]:
#pip install -U spacy #spacy in /usr/local/lib/python3.6/dist-packages (2.2.4)   1 time
#!pip install spacy
#!python -m spacy download es          #Every time it reload    ,   Successfully installed es-core-news-sm-2.2.5

import spacy
nlp = spacy.load('es')
#nlp = spacy.load("en_core_web_sm")  #Do es not work
print('spacy.es loaded')

#!pip install spacy-lookup     
#!pip install spacy-lookup    #For Dictionary based NER ,  Successfully installed flashtext-2.7 spacy-lookup-0.1.0
from spacy_lookup import Entity

from spacy import displacy


"""Spanish md
#!pip install https://github.com/explosion/spacy-models/releases/download/es_core_news_md-2.2.5/es_core_news_md-2.2.5.tar.gz
!python -m spacy download es_core_news_md
import spacy
nlp = spacy.load("es_core_news_md")           # load model package https://spacy.io/models/es"""

spacy.es loaded


In [0]:
"""Loop for one text file: S0004-06142005000700014-1 is called text, wich is going to de document"""

for index, row in df_train_articles.iterrows():
    #print(index)
    #print(row[0])
    #print(row[1])
    if index == 0:
      text = row[1]
      print(text)

Describimos el caso de un varón de 37 años con vida previa activa que refiere dolores osteoarticulares de localización variable en el último mes y fiebre en la última semana con picos (matutino y vespertino) de 40 C las últimas 24-48 horas, por lo que acude al Servicio de Urgencias. Antes de comenzar el cuadro estuvo en Extremadura en una región endémica de brucella, ingiriendo leche de cabra sin pasteurizar y queso de dicho ganado. Entre los comensales aparecieron varios casos de brucelosis. Durante el ingreso para estudio del síndrome febril con antecedentes epidemiológicos de posible exposición a Brucella presenta un cuadro de orquiepididimitis derecha.
La exploración física revela: Tª 40,2 C; T.A: 109/68 mmHg; Fc: 105 lpm. Se encuentra consciente, orientado, sudoroso, eupneico, con buen estado de nutrición e hidratación. En cabeza y cuello no se palpan adenopatías, ni bocio ni ingurgitación de vena yugular, con pulsos carotídeos simétricos. Auscultación cardíaca rítmica, sin soplos

###Filtering

In [0]:
#text = [s.replace('&quot;', '\'') for s in text]
#print(text)

###Sentence splitting

Spacy also returns the list of sentences in the text. The list of sentences is stored in the property **sents**. For each sentence, you can traverse its tokens and read their properties. 

In the following cell, for each token in a sentence, we show token and its PoS tag :

In [0]:
document = nlp(text)
for i, sent in enumerate(document.sents):      #divide document into sentences
    print(i, sent.text)
    print(sent.start_char, sent.end_char)    #start_char and end_char are the position of the first and last character of sentence respecting the document.
    for token in sent:
      print('\t',token.orth_, token.pos_)
    print("----------------------------------------------------------------------------------------------------------------------------")

### Case-folding
A considerar:
* Se introduce ambigüedad, e.g. 'Apple' y 'apple'.
* Afecta a acrónimos, siglas y abreviaciones.

In [0]:
#text = [e.lower() for e in text]
#print(text)

###Tokenization


Once you have loaded a document, you can parser it using the Spaci. You only have to pass the document as argument to the object **nlp**, which was created above. This return a list of objects, which represent the tokens of the text and their properties. These properties are very useful in the representation of instances for NLP tasks such as Named Entity Recognition or Text Clasification. 

In the next cell, we iterate the tokens and show some of their properties :
-    original text
-    shape (which is a pattern of its uppercase and lowercase)
-  PoS tag
-   lemma of token
-    prefix
-    sufix
-    Brown cluster id: cluster which the token belongs

You can find the description of the token properties at the following link: https://spacy.io/api/token#attributes


* Cómo funciona: [spaCy](https://spacy.io/usage/linguistic-features#tokenization)
* Excepciones en español para spaCy: [GitHub](https://github.com/explosion/spaCy/blob/master/spacy/lang/es/tokenizer_exceptions.py)

https://colab.research.google.com/drive/1Vf2b9jSzxyQGJAG00G_mqBxRCs3VvviJ#scrollTo=E2owzCAJFhBH

In [0]:
for i, token in enumerate(document):
    print("original:", token.orth_)
    print("shape:", token.shape_)
    print("PoS tag:", token.pos_)


    #print("lowercased:", token.lower_)
    print("lemma:", token.lemma_)
    print("prefix:", token.prefix_)
    print("suffix:", token.suffix_)
    print("Brown cluster id:", token.cluster)
    print("---------------------------------------------------------------------------------------------------------------------------")
    #only shows 5 tokens
    if i > 10:
        break

### Stopwords filtering

In [0]:
spacy_stopwords = spacy.lang.es.stop_words.STOP_WORDS
print('Number of stop words: %d' % len(spacy_stopwords))
print('First ten stop words: %s' % list(spacy_stopwords)[:10])

Number of stop words: 551
First ten stop words: ['menos', 'segunda', 'intento', 'al', 'mios', 'tenemos', 'sera', 'antano', 'primeros', 'buenas']


### Entity Detection


Spacy is also able to identify the named entities that occur in the text. The named entities contained in the document are stored in its property ents: For each entity, you can acces the following properties:

-  string: contains the whole mention of the named entity.
-  label: is the entity type.
-  start_char and end_char are the offsets of the mention in the text.


In [0]:
for i,sent in enumerate(document.sents):
  print("Text sentence: ", i, sent)
  print('Named entities for the sentence: ', i)
  for entity in sent.ents:
    print('\t',entity.string,entity.label_,entity.start_char, entity.end_char)
  print("-----------------------------------------------------------------------------------------------------------------------------") 


Text sentence:  0 Describimos el caso de un varón de 37 años con vida previa activa que refiere dolores osteoarticulares de localización variable en el último mes y fiebre en la última semana con picos (matutino y vespertino) de 40 C las últimas 24-48 horas, por lo que acude al Servicio de Urgencias.
Named entities for the sentence:  0
	 dolores osteoarticulares  DIAGNOSTICO 78 102
	 fiebre  DIAGNOSTICO 147 153
-----------------------------------------------------------------------------------------------------------------------------
Text sentence:  1 Antes de comenzar el cuadro estuvo en Extremadura en una región endémica de brucella, ingiriendo leche de cabra sin pasteurizar y queso de dicho ganado.
Named entities for the sentence:  1
	 brucella DIAGNOSTICO 360 368
-----------------------------------------------------------------------------------------------------------------------------
Text sentence:  2 Entre los comensales aparecieron varios casos de brucelosis.
Named entities f

In [0]:
for ent in document.ents:
  print(ent.string,ent.label_)

dolores osteoarticulares  DIAGNOSTICO
fiebre  DIAGNOSTICO
brucella DIAGNOSTICO
brucelosis DIAGNOSTICO
síndrome febril  DIAGNOSTICO
exposición a Brucella  DIAGNOSTICO
orquiepididimitis  DIAGNOSTICO
adenopatías DIAGNOSTICO
bocio  DIAGNOSTICO
ingurgitación de vena yugular DIAGNOSTICO
soplos DIAGNOSTICO
varices  DIAGNOSTICO
edemas DIAGNOSTICO
teste derecho aumentado de tamaño DIAGNOSTICO
doloroso  DIAGNOSTICO
epidídimo DIAGNOSTICO
mm  DIAGNOSTICO
Brucella  DIAGNOSTICO
Brucella DIAGNOSTICO
Rx tórax PROCEDIMIENTO
Ecografía abdominal PROCEDIMIENTO
TAC craneal PROCEDIMIENTO
Ecocardiograma transtorácico PROCEDIMIENTO
Ecografía testicular PROCEDIMIENTO
orquiepididimitis  DIAGNOSTICO
Brucella  DIAGNOSTICO


With the start and end of the entities in every sentence

In [0]:
for i,s in enumerate(document.sents):
  print("---------------------------------------------------------------------------------------------------------------------------")
  print("Sentence text: ", i, s)
  
  start_sentence_text = s.start_char
  #print("start_sentence_text:", start_sentence_text)
  end_sentence_text = s.end_char
  #print("end_sentence_text:", end_sentence_text)
  
  for e in s.ents:
    start_word_text = e.start_char    #entity begin respect with document
    #print("start_word_text:", start_word_text)
    end_word_text = e.end_char        #entity end respect with document
    #print("end_word_text:", end_word_text)
    start_word_sentence = start_word_text - start_sentence_text    #entity begin respect with sentence
    end_word_sentence = end_word_text - start_sentence_text 
    print('\t',"Entity text reference:", e.string,"\t---  start_word_sentence:", start_word_sentence, "\t---  end_word_sentence:",  end_word_sentence)

---------------------------------------------------------------------------------------------------------------------------
Sentence text:  0 Describimos el caso de un varón de 37 años con vida previa activa que refiere dolores osteoarticulares de localización variable en el último mes y fiebre en la última semana con picos (matutino y vespertino) de 40 C las últimas 24-48 horas, por lo que acude al Servicio de Urgencias.
	 Entity text reference: dolores osteoarticulares  	---  start_word_sentence: 78 	---  end_word_sentence: 102
	 Entity text reference: fiebre  	---  start_word_sentence: 147 	---  end_word_sentence: 153
---------------------------------------------------------------------------------------------------------------------------
Sentence text:  1 Antes de comenzar el cuadro estuvo en Extremadura en una región endémica de brucella, ingiriendo leche de cabra sin pasteurizar y queso de dicho ganado.
	 Entity text reference: brucella 	---  start_word_sentence: 76 	---  end_wo

###Noun Chunker


You can obtain the list of noun phrases in a sentence directly using the property **noun_chunks**. 

For each noun phrase, it is possible to return the following properties:
- text: the whole text of the noun phrase
- root: the root of the noun phrase
- dep: the grammatical relationship of the noun phrase in in the sentence. You can find more information about these relationships (dependencies) at https://nlp.stanford.edu/software/dependencies_manual.pdf. Some of these dependencies are:
 - *nsubj*: it is the syntactic subject. For example, 'Clinton defeated Dole', nsubj(defeated,Clinton)
 - *dobj*:  it is the direct object of a VP. For example,  'She gave me a raise' - > dobj (gave, raise).
 - *pobj*: it is the object of a preposition  (for example, 'I sit on the chair' -> pobj(on, chair)
- head: it represent the source of the grammatical relationship. For example, the relationship nsubj between shift and Autonomus cars.  
 


In [0]:
doc = nlp(text)

for chunk in doc.noun_chunks:
    print('text chunk:',chunk.text)
    print('root chunk:',chunk.root.text)
    print('grammatical dependency:',chunk.root.dep_)
    print('head chunk:',chunk.root.head.text)
    print('---------------------------------')

text chunk: el caso
root chunk: caso
grammatical dependency: obj
head chunk: Describimos
---------------------------------
text chunk: un varón
root chunk: varón
grammatical dependency: nmod
head chunk: caso
---------------------------------
text chunk: años
root chunk: años
grammatical dependency: nmod
head chunk: varón
---------------------------------
text chunk: vida
root chunk: vida
grammatical dependency: nmod
head chunk: varón
---------------------------------
text chunk: que
root chunk: que
grammatical dependency: nsubj
head chunk: refiere
---------------------------------
text chunk: dolores osteoarticulares
root chunk: dolores osteoarticulares
grammatical dependency: obj
head chunk: refiere
---------------------------------
text chunk: localización
root chunk: localización
grammatical dependency: nmod
head chunk: dolores osteoarticulares
---------------------------------
text chunk: el último mes
root chunk: mes
grammatical dependency: obl
head chunk: refiere
----------------

###Dependency parsing

SpaCy provides a fast and accurate syntactic dependency parser. This parser obtains the grammatical relations between tokens in a sentence. This information is crucial for NLP task such as relation extraction. 

You can try to parse any sentence using  the online tool http://nlp.stanford.edu:8080/corenlp/process


The following cell shows the dependency parsing for a sentence. For each token, you can determine 

In [0]:
parsedEx = nlp(text)
# shown as: original token, dependency tag, head word
for token in parsedEx:
    print("word:",token.orth_)
    print("grammatical relation:", token.dep_)
    print("connected word (head):", token.head.orth_)
    print('------------------------------------------')

word: Describimos
grammatical relation: ROOT
connected word (head): Describimos
------------------------------------------
word: el
grammatical relation: det
connected word (head): caso
------------------------------------------
word: caso
grammatical relation: obj
connected word (head): Describimos
------------------------------------------
word: de
grammatical relation: case
connected word (head): varón
------------------------------------------
word: un
grammatical relation: det
connected word (head): varón
------------------------------------------
word: varón
grammatical relation: nmod
connected word (head): caso
------------------------------------------
word: de
grammatical relation: case
connected word (head): años
------------------------------------------
word: 37
grammatical relation: nummod
connected word (head): años
------------------------------------------
word: años
grammatical relation: nmod
connected word (head): varón
------------------------------------------
word:

In [0]:
parsedEx = nlp(text)
for i, sent in enumerate(parsedEx.sents):      #divide document into sentences
    #print(i, sent.text)
    doc = nlp(sent.text)
    displacy.render(doc, jupyter=True, style="dep")
    print("----------------------------------------------------------------------------------------------------------------------------")   

----------------------------------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------------------------------


...

----------------------------------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------------------------------


###Coreference

In [0]:
#!pip3 install neuralcoref
#import neuralcoref


"""Falta código"""

###NER extraction with Displacy

Using displaCy in Jupyter notebooks. Reference in : https://spacy.io/usage/visualizers

For dictionary based NER, it is necessary spacy-lookup library.

spacy-lookup library:   Github.

https://github.com/mpuig/spacy-lookup

Notebook.

https://colab.research.google.com/drive/1Xsne2tMz2IDk0X2Gn-9GIUcVDpAfS3-W?authuser=1#scrollTo=MR_FqfiQ5cLx


#### Automatic NER with Displacy using entities from Spacy

Divide the article in sentences with Document.ents 

*  Type : label
*  Value : text_refence
*  Start_character
*  End_character


In [0]:
document = nlp(text)
#print('Original Sentence: %s' % (text))
for entity in document.ents:
    print('Type: {}, Value: {}, star: {}, end: {}'.format(entity.label_, entity.text,entity.start_char, entity.end_char))

document = nlp(str(text))
displacy.render(document, jupyter=True, style="ent")    #NER
#displacy.render(document, jupyter=True, style="dep")   #dependency parsing

Type: MISC, Value: C, star: 214, end: 215
Type: ORG, Value: Servicio de Urgencias, star: 261, end: 282
Type: LOC, Value: Extremadura, star: 322, end: 333
Type: MISC, Value: Entre los comensales, star: 437, end: 457
Type: MISC, Value: Durante el ingreso, star: 498, end: 516
Type: PER, Value: Brucella, star: 607, end: 615
Type: MISC, Value: La exploración física revela, star: 665, end: 693
Type: MISC, Value: Tª 40,2 C, star: 695, end: 704
Type: MISC, Value: T.A, star: 706, end: 709
Type: MISC, Value: mmHg, star: 718, end: 722
Type: MISC, Value: Se encuentra consciente, star: 737, end: 760
Type: MISC, Value: En cabeza y cuello, star: 837, end: 855
Type: MISC, Value: Auscultación cardíaca rítmica, star: 959, end: 988
Type: PER, Value: Auscultación, star: 1023, end: 1035
Type: PER, Value: Abdomen, star: 1086, end: 1093
Type: MISC, Value: En la exploración neurológica, star: 1137, end: 1166
Type: PER, Value: Extremidades, star: 1222, end: 1234
Type: PER, Value: Pulsos, star: 1258, end: 1264


####Dictionary based NER with Displacy using entities from AnnotationsD and AnnotationsP

In [0]:
def load_dictionary(df):
  """This function reads a df and save the fourth column into a list. 
  This list is an input parameter, which is modified. We need to pass
  the list as parameter, because if the list is returned, 
  its type becomes to NoneType"""
  dictionary=[]
  for index, row in df.iterrows():
    dictionary.append(row[3].lower())
  print(dictionary)
  print('length of the dictionary loaded: ', len(dictionary))

  return(dictionary)

path_codiesp='/content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/'

print("---Load dictionary 4. Annotations, Diagnostics, train")
train_diagnostics_dictionary = load_dictionary(df_train_annotationsD)
#print(train_diagnostics)

print("---Load dictionary 5. Annotations, Diagnostics, dev")
dev_diagnostics_dictionary = load_dictionary(df_dev_annotationsD)
#print(dev_diagnostics)

print("---Load dictionaty 7. Annotations, Procedures, train")
train_procedures_dictionary = load_dictionary(df_train_annotationsP)
#print(train_procedures)

print("---Load dictionaty 8. Annotations, Procedures, dev")
dev_procedures_dictionary = load_dictionary(df_dev_annotationsP)
#print(dev_procedures)

---Load dictionary 4. Annotations, Diagnostics, train
['bocio', 'brucella', 'brucella', 'febril', 'dolores', 'edemas', 'exposición a brucella', 'brucella', 'teste derecho aumentado de tamaño', 'ingurgitación de vena yugular', 'brucella', 'orquiepididimitis', 'orquiepididimitis', 'varices', 'dolores osteoarticulares', 'fiebre', 'síndrome febril', 'leiomioma vesical', 'sangrado', 'rabdomiosarcoma pleomórfico del cordón espermático', 'pérdida de peso', 'dolorosa', 'tumor paratesticular izquierdo', 'tumorales cordón espermático izquierdo', 'cordón espermático izquierdo neoplásica', 'retroperitoneal masa', 'doloroso', 'cordón espermático doloroso', 'testes masa', 'masa testes', 'porta sonda vesical', 'paraparesia espástica tropical', 'hepatocarcinoma', 'hipertensión portal', 'cirrosis hepática de probable origen alcohólico', 'htlv-i positiva', 'ascitis', 'aumento de los valores de transaminasas', 'porta sonda vesical permanente', 'trasplantectomía', 'hda secundaria a varices esofágicas', 'd

-  **Add Named Entities metadata to Doc objects in Spacy.** First, we load the model and replace the NER module with de entity diagnosticEnt. We do this to avoid overlapping of entities. Then, we also add procedureEnt.

In [0]:
diagnosticEnt = Entity(keywords_list=train_diagnostics_dictionary,label="DIAGNOSTICO")
print(type(diagnosticEnt))

procedureEnt = Entity(keywords_list=train_procedures_dictionary,label="PROCEDIMIENTO")
print(type(procedureEnt))

#We replace the common entities with diagnostics
nlp.replace_pipe("ner", diagnosticEnt)
#We add pipe
nlp.add_pipe(procedureEnt, last=True)

print('entities loaded in nlp')

<class 'spacy_lookup.Entity'>
<class 'spacy_lookup.Entity'>
entities loaded in nlp


Pipeline name: ['tagger', 'parser', 'ner', 'entity']

- **Detect Named Entities using dictionaries.** We can process a text and show its entities.

In [0]:
"""TEXT FILE S0004-06142005000700014-1 :
Describimos el caso de un varón de 37 años con vida previa activa que refiere dolores osteoarticulares de localización variable en el último mes y fiebre en la última semana con picos (matutino y vespertino) de 40 C las últimas 24-48 horas, por lo que acude al Servicio de Urgencias. Antes de comenzar el cuadro estuvo en Extremadura en una región endémica de brucella, ingiriendo leche de cabra sin pasteurizar y queso de dicho ganado. Entre los comensales aparecieron varios casos de brucelosis. Durante el ingreso para estudio del síndrome febril con antecedentes epidemiológicos de posible exposición a Brucella presenta un cuadro de orquiepididimitis derecha.
La exploración física revela: Tª 40,2 C; T.A: 109/68 mmHg; Fc: 105 lpm. Se encuentra consciente, orientado, sudoroso, eupneico, con buen estado de nutrición e hidratación. En cabeza y cuello no se palpan adenopatías, ni bocio ni ingurgitación de vena yugular, con pulsos carotídeos simétricos. Auscultación cardíaca rítmica, sin soplos, roces ni extratonos. Auscultación pulmonar con conservación del murmullo vesicular. Abdomen blando, depresible, sin masas ni megalias. En la exploración neurológica no se detectan signos meníngeos ni datos de focalidad. Extremidades sin varices ni edemas. Pulsos periféricos presentes y simétricos. En la exploración urológica se aprecia el teste derecho aumentado de tamaño, no adherido a piel, con zonas de fluctuación e intensamente doloroso a la palpación, con pérdida del límite epidídimo-testicular y transiluminación positiva.
Los datos analíticos muestran los siguentes resultados: Hemograma: Hb 13,7 g/dl; leucocitos 14.610/mm3 (neutrófilos 77%); plaquetas 206.000/ mm3. VSG: 40 mm 1ª hora. Coagulación: TQ 87%; TTPA 25,8 seg. Bioquímica: Glucosa 117 mg/dl; urea 29 mg/dl; creatinina 0,9 mg/dl; sodio 136 mEq/l; potasio 3,6 mEq/l; GOT 11 U/l; GPT 24 U/l; GGT 34 U/l; fosfatasa alcalina 136 U/l; calcio 8,3 mg/dl. Orina: sedimento normal.
Durante el ingreso se solicitan Hemocultivos: positivo para Brucella y Serologías específicas para Brucella: Rosa de Bengala +++; Test de Coombs > 1/1280; Brucellacapt > 1/5120. Las pruebas de imagen solicitadas ( Rx tórax, Ecografía abdominal, TAC craneal, Ecocardiograma transtorácico) no evidencian patología significativa, excepto la Ecografía testicular, que muestra engrosamiento de la bolsa escrotal con pequeña cantidad de líquido con septos y testículo aumentado de tamaño con pequeñas zonas hipoecoicas en su interior que pueden representar microabscesos.
Con el diagnóstico de orquiepididimitis secundaria a Brucella se instaura tratamiento sintomático (antitérmicos, antiinflamatorios, reposo y elevación testicular) así como tratamiento antibiótico específico: Doxiciclina 100 mg vía oral cada 12 horas (durante 6 semanas) y Estreptomicina 1 gramo intramuscular cada 24 horas (durante 3 semanas). El paciente mejora significativamente de su cuadro tras una semana de ingreso, decidiéndose el alta a su domicilio donde completó la pauta de tratamiento antibiótico. En revisiones sucesivas en consultas se constató la completa remisión del cuadro.
"""

"""DIAGNOSTICS
|    0 | S0004-06142005000700014-1  | DIAGNOSTICO | e04.9 (automatic suggestion) | bocio                                                                                                             |
|    1 | S0004-06142005000700014-1  | DIAGNOSTICO | a23.9                        | Brucella                                                                                                          |
|    2 | S0004-06142005000700014-1  | DIAGNOSTICO | a23.9                        | Brucella                                                                                                          |
|    3 | S0004-06142005000700014-1  | DIAGNOSTICO | r50.9                        | febril                                                                                                            |
|    4 | S0004-06142005000700014-1  | DIAGNOSTICO | r52                          | dolores                                                                                                           |
|    5 | S0004-06142005000700014-1  | DIAGNOSTICO | r60.9                        | edemas                                                                                                            |
|    6 | S0004-06142005000700014-1  | DIAGNOSTICO | z20.818                      | exposición a Brucella                                                                                             |
|    7 | S0004-06142005000700014-1  | DIAGNOSTICO | a23.9                        | Brucella                                                                                                          |
|    8 | S0004-06142005000700014-1  | DIAGNOSTICO | n44.8                        | teste derecho aumentado de tamaño                                                                                 |
|    9 | S0004-06142005000700014-1  | DIAGNOSTICO | i87.8                        | ingurgitación de vena yugular                                                                                     |
|   10 | S0004-06142005000700014-1  | DIAGNOSTICO | a23.9                        | brucella                                                                                                          |
|   11 | S0004-06142005000700014-1  | DIAGNOSTICO | n45.3                        | orquiepididimitis                                                                                                 |
|   12 | S0004-06142005000700014-1  | DIAGNOSTICO | n45.3                        | orquiepididimitis                                                                                                 |
|   13 | S0004-06142005000700014-1  | DIAGNOSTICO | i83.90                       | varices                                                                                                           |
|   14 | S0004-06142005000700014-1  | DIAGNOSTICO | m25.50                       | dolores osteoarticulares                                                                                          |
|   15 | S0004-06142005000700014-1  | DIAGNOSTICO | r50.9                        | fiebre                                                                                                            |
|   16 | S0004-06142005000700014-1  | DIAGNOSTICO | r50.9                        | síndrome febril      


PROCEDURES
|    0 | S0004-06142005000700014-1  | PROCEDIMIENTO | bw03zzz                        | Rx tórax                                                                                                                                |
|    1 | S0004-06142005000700014-1  | PROCEDIMIENTO | bn20                           | TAC craneal                                                                                                                             |
|    2 | S0004-06142005000700014-1  | PROCEDIMIENTO | bw40zzz                        | Ecografía abdominal                                                                                                                     |
|    3 | S0004-06142005000700014-1  | PROCEDIMIENTO | bv44zzz                        | Ecografía testicular                                                                                                                    |
|    4 | S0004-06142005000700014-1  | PROCEDIMIENTO | 3E02329                        | Estreptomicina intramuscular"""  

In [0]:
#text = text.lower() #en el caso de poner todo minusculas
document = nlp(text)

#print([(token.text, token._.canonical) for token in document if token._.is_entity])  #en el caso de querer normalizar las entidades [('dolores osteoarticulares', 'dolores osteoarticulares'),... ('exposición a Brucella', 'exposición a brucella'), ... ('Brucella', 'brucella')]

#dict([(str(x), x.label_) for x in nlp(str(text)).ents])  #dictionary with entities and types {'Brucella': 'DIAGNOSTICO',... 'varices': 'DIAGNOSTICO'}

#print('Original Sentence: %s' % (text))

for entity in document.ents:                        #divide document into entities
    print('Type: {}, Value: {}, star: {}, end: {}'.format(entity.label_, entity.text,entity.start_char, entity.end_char))   #start_char and end_char are the position of the first and last character of an entity, respect the document begining.

ents=["DIAGNOSTICO","PROCEDIMIENTO"]
colors={"DIAGNOSTICO":"#9FDEF9","PROCEDIMIENTO":"#BA9FF9"}    #blue and purple
options = {"ents": ents, "colors": colors}

displacy.render(document, jupyter=True, style="ent", options=options)
#displacy.render(document, jupyter=True, style="ent", options=get_entity_options(ents))


####Dictionary based NER with Displacy separated by sentences

In [0]:
ents=["DIAGNOSTICO","PROCEDIMIENTO"]
colors={"DIAGNOSTICO":"#9FDEF9","PROCEDIMIENTO":"#BA9FF9"}    #blue and purple
options = {"ents": ents, "colors": colors}


document = nlp(text)
for i, sent in enumerate(document.sents):      #divide document into sentences
    #print(i, sent.text)
    print(sent.start_char, sent.end_char)    #start_char and end_char are the position of the first and last character of sentence respecting the document.
    #for entity in doc.ents:                        #divide document into entities
      #print('Type: {}, Value: {}, star: {}, end: {}'.format(entity.label_, entity.text,entity.start_char, entity.end_char) 
    doc = nlp(sent.text)
    displacy.render(doc, jupyter=True, style="ent", options=options)
    #for token in sent:                        #divide sentences into tokens
      #print('\t',token.orth_, token.pos_)   #	ID of the verbatim text content, 	Coarse-grained part-of-speech.
    print("----------------------------------------------------------------------------------------------------------------------------")

0 283


----------------------------------------------------------------------------------------------------------------------------
284 436


----------------------------------------------------------------------------------------------------------------------------
437 497


----------------------------------------------------------------------------------------------------------------------------
498 665


----------------------------------------------------------------------------------------------------------------------------
665 736


...

----------------------------------------------------------------------------------------------------------------------------
2859 3025


/usr/lib/python3.6/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


----------------------------------------------------------------------------------------------------------------------------
3026 3108


/usr/lib/python3.6/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


----------------------------------------------------------------------------------------------------------------------------


#Output

version 1.0 

- Sentence splitting
- Tokenization
- Entity Detection
- Noun Chunker
- Dependency parsing
- Ner extraction with Displacy by sentences with dictionary (annotations)



